In [1]:
# Load the training dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
from dotenv import load_dotenv

load_dotenv()
raw_data_path = os.getenv('RAW_DATA')
train_data_path = os.getenv('TRAINING_DATA')
nat_val_data_path = os.getenv('NATIONAL_VAL')
intl_val_data_path = os.getenv('INTL_VAL')
intl_val_incl_sex_data_path = os.getenv('INTL_VAL_INCL_SEX')

train = pd.read_csv(train_data_path)
df = pd.read_csv(raw_data_path)

In [2]:
print('Overall dataset:\n' + 'Number of participants: ' + str(df.shape[0]) + '\nNumber of features: ' + str(df.shape[1]))

Overall dataset:
Number of participants: 23742
Number of features: 102


## Describe the training data

In [3]:
fu_desc = round(pd.DataFrame(train["esrd_lower"].to_numpy()[np.where(train["ESRD"] == 1)])[0].describe(), 1)
lower_fu = round(np.median(train["esrd_lower"].to_numpy()[np.where(train["ESRD"] == 1)]), 1)
print('Training data:\n' + 'Number of participants: ' + str(train.shape[0]) + '\nNumber of features: ' + str(train.shape[1]))
print(f'Number of events: {(train["esrd_lower"] == train["esrd_upper"]).sum()} ({round(((train["esrd_lower"] == train["esrd_upper"]).sum() / train.shape[0]) * 100, 1)}%)')
print(f'Median time to kidney failure (uncensored): {fu_desc["50%"]} [IQR, {fu_desc["25%"]}-{fu_desc["75%"]}] months')

# Follow-up
fu_desc = round(train['esrd_lower'].describe(), 1)
print(f'Follow-up: {fu_desc["50%"]} [IQR, {fu_desc["25%"]} - {fu_desc["75%"]}] months\n')
print('Characteristics:')
# 0 represents females
print(f'Females: {(train["Sesso"] == 0).sum()} ({round(((train["Sesso"] == 0).sum() / train.shape[0]) * 100, 1)}%)')
# Age
age_desc = round(train['Eta1'].describe(), 1)
print(f'Age: {age_desc["50%"]} [IQR, {age_desc["25%"]} - {age_desc["75%"]}] years')
# eGFR
egfr_desc = round(train['Epi1'].describe(), 1)
print(f'eGFR: {egfr_desc["50%"]} [IQR, {egfr_desc["25%"]} - {egfr_desc["75%"]}]\n')
# Urinary protein excretion
prot_excr_desc = round(train['Prot1'].describe(), 1)
print(f'Urinary protein excretion: {prot_excr_desc["50%"]} [IQR, {prot_excr_desc["25%"]} - {prot_excr_desc["75%"]}] g/24hrs\n')
print('Features:')
print(f'{train.columns.values}')

Training data:
Number of participants: 3599
Number of features: 20
Number of events: 728 (20.2%)
Median time to kidney failure (uncensored): 25.9 [IQR, 14.0-40.6] months
Follow-up: 37.3 [IQR, 25.2 - 54.9] months

Characteristics:
Females: 1434 (39.8%)
Age: 70.1 [IQR, 60.6 - 76.6] years
eGFR: 32.3 [IQR, 22.0 - 42.4]

Urinary protein excretion: 0.4 [IQR, 0.1 - 1.2] g/24hrs

Features:
['Studio' 'Sesso' 'Diabete' 'ESRD' 'Eta1' 'Prot1' 'Creat1' 'Epi1'
 'Colesterolo1' 'Hb1' 'Ca1' 'P1' 'BMI1' 'ckd_cause_hypertens'
 'ckd_cause_diabet' 'ckd_cause_glom_dis' 'ckd_cause_tubul_inter'
 'ckd_cause_pkd' 'esrd_upper' 'esrd_lower']


In [4]:
def baseline_table(features, data_frame, features_dict = None):
    means = []
    sds = []
    lowers = []
    medians = []
    uppers = []
    
    for i, feat in enumerate(features):
        description = round(data_frame[feat].describe(), 1)
        means.append(description['mean'])
        sds.append(description['std'])
        lowers.append(description['25%'])
        medians.append(description['50%'])
        uppers.append(description['75%'])
    if features_dict != None:
        feat_names = list()
        for feat in features:
            feat_names.append(feature_dict[feat])
        table_1 = {'Feature': feat_names, 'Mean': means, 'Standard deviation': sds, '25%': lowers, '50%': medians, '75%': uppers}
    else: 
        table_1 = {'Mean': means, 'Standard deviation': sds, '25%': lowers, '50%': medians, '75%': uppers}
    return pd.DataFrame(table_1, index = features)

In [5]:
features = ['Eta1', 'Epi1', 'Prot1', 'Colesterolo1', 
            'Hb1', 'Ca1', 'P1', 'BMI1']
feature_dict = {'Creat1': 'Creatinine',
                'Epi1': 'eGFR (mL/min/1.73m$^{2}$)',
                'Prot1': 'Proteinuria (g/day)',
                'Albumina1': 'Serum albumin',
                'Colesterolo1': 'Total cholesterol (mg/dL)',
                'P1': 'Plasma phosphorus (mg/dL)',
                'BMI1': 'Body mass index (kg/m$^{2}$)',
                'Hb1': 'Hemoglobin (g/dL)',
                'Ldl1': 'Low density lipoprotein',
                'Ca1': 'Plasma calcium (mg/dL)',
                'Glicemia1': 'Glucose (mg/dL)',
                'Eta1': 'Age (years)',
                'Sesso': 'Sex',
                'Ca1': 'Plasma Calcium (mg/dL)',
                'Sodiuria1': 'Urinary sodium',
                'Trigliceridi1': 'Triglycerides (mg/dL)',
                'ckd_cause_unknown': 'Unknown cause CKD',
                'Angina': 'Angina complaints',
                'IMA': 'Myocardial infarction',
                'Pas1': 'Systolic blood pressure',
                'Pad1': 'Peripheral artery disease',
                'ckd_cause_diabet': 'CKD cause diabetes',
                'ckd_cause_hypertens': 'CKD cause hypertension',
                'ckd_cause_tubul_inter': 'CKD cause tubulointerstitial disease',
                'ckd_cause_pkd': 'CKD cause ADPKD',
                'CHF': 'Congestive heart failure',
                'PVD': 'Peripheral vascular disease',
                'CVD': 'Cardiovascular disease'}

table_1 = baseline_table(features = features, data_frame = train, features_dict = feature_dict)
table_1

,Feature,Mean,Standard deviation,25%,50%,75%
Eta1,Age (years),67.4,13.0,60.6,70.1,76.6
Epi1,eGFR (mL/min/1.73m$^{2}$),32.5,13.0,22.0,32.3,42.4
Prot1,Proteinuria (g/day),1.1,1.8,0.1,0.4,1.2
Colesterolo1,Total cholesterol (mg/dL),195.0,42.7,168.0,193.7,218.0
Hb1,Hemoglobin (g/dL),12.5,1.8,11.4,12.5,13.8
Ca1,Plasma Calcium (mg/dL),9.3,0.6,9.0,9.3,9.7
P1,Plasma phosphorus (mg/dL),3.8,0.8,3.3,3.7,4.1
BMI1,Body mass index (kg/m$^{2}$),27.7,4.8,24.5,27.2,30.1


## Describe the national validation set

In [6]:
# Import the results of the feature selection
import json

with open('raw_results/feature_selection_results.json') as f:
    var_dict = json.load(f)

df_feat_import = pd.DataFrame({'feat': var_dict['feat'], 'c_index': var_dict['c_index']})
selected_feat = list(df_feat_import['feat'])[:np.argmax(df_feat_import['c_index']) + 3]
selected_feat

test = pd.read_csv(nat_val_data_path)
test = test.loc[test['Studio'] == 7]
test = test[list(selected_feat + ['Sesso', 'esrd_lower', 'esrd_upper'])]
test.dropna(axis = 0, how = 'any', inplace = True)
test.shape

(346, 18)

In [7]:
print('National validation:\n')
print(f'Number of features: {test.shape[1]}')
print(f'Number of participants: {test.shape[0]}')
print(f'Number of events: {(test["esrd_lower"] == test["esrd_upper"]).sum()} ({round(((test["esrd_lower"] == test["esrd_upper"]).sum() / test.shape[0]) * 100, 1)}%)')
# Follow-up
fu_desc = round(test['esrd_lower'].describe(), 1)
print(f'Follow-up: {fu_desc["50%"]} [IQR, {fu_desc["25%"]} - {fu_desc["75%"]}] months\n')
print('Characteristics:')
# 0 represents females
print(f'Females: {(test["Sesso"] == 0).sum()} ({round(((test["Sesso"] == 0).sum() / test.shape[0]) * 100, 1)}%)')
# Age
age_desc = round(test['Eta1'].describe(), 1)
print(f'Age: {age_desc["50%"]} [IQR, {age_desc["25%"]} - {age_desc["75%"]}] years')
# eGFR
egfr_desc = round(test['Epi1'].describe(), 1)
print(f'eGFR: {egfr_desc["mean"]} ± {egfr_desc["std"]}\n')
print('Features:')
print(f'{test.columns.values}')

National validation:

Number of features: 18
Number of participants: 346
Number of events: 129 (37.3%)
Follow-up: 51.5 [IQR, 25.1 - 75.2] months

Characteristics:
Females: Sesso    159
Sesso    159
dtype: int64 (Sesso    46.0
Sesso    46.0
dtype: float64%)
Age: 61.6 [IQR, 47.6 - 69.4] years
eGFR: 31.7 ± 14.4

Features:
['Epi1' 'Prot1' 'BMI1' 'Eta1' 'P1' 'Ca1' 'Hb1' 'Colesterolo1'
 'ckd_cause_hypertens' 'Sesso' 'Diabete' 'ckd_cause_pkd'
 'ckd_cause_tubul_inter' 'ckd_cause_glom_dis' 'ckd_cause_diabet' 'Sesso'
 'esrd_lower' 'esrd_upper']


## Describe the international validation set

In [8]:
test_intl = pd.read_csv(intl_val_data_path)

test_intl = test_intl.merge(pd.read_csv(intl_val_incl_sex_data_path), on='ID_UMCG')
test_intl.columns
test_intl = test_intl.drop(columns = 'ID_UMCG')
test_intl['GESLACHT'] = test_intl['GESLACHT'].replace(1, 'female') # for the check see the preprocessing R file
test_intl['GESLACHT'] = test_intl['GESLACHT'].replace(2, 'male')
test_intl.dropna(axis = 0, how = 'any', inplace = True)

In [9]:
print('International validation:\n')
print(f'Number of features: {test_intl.shape[1]}')
print(f'Number of participants: {test_intl.shape[0]}')
print(f'Number of events: {(test_intl["esrd_lower"] == test_intl["esrd_upper"]).sum()} ({round(((test_intl["esrd_lower"] == test_intl["esrd_upper"]).sum() / test_intl.shape[0]) * 100, 1)}%)\n')
print('Characteristics:')
print(f'Females: {(test_intl["GESLACHT"] == "female").sum()} ({round(((test_intl["GESLACHT"] == "female").sum() / test_intl.shape[0]) * 100, 1)}%)')
print('Features:')
print(f'{test_intl.columns.values}')

International validation:

Number of features: 11
Number of participants: 297
Number of events: 114 (38.4%)

Characteristics:
Females: 95 (32.0%)
Features:
['Epi1' 'Proteinurieurine' 'Fosfaat' 'bmi' 'Hbmmolperliter' 'Calciumbloed'
 'totaalcholesterol' 'age' 'esrd_lower' 'esrd_upper' 'GESLACHT']


In [10]:
baseline_table(features = test_intl.columns[:len(test_intl.columns) - 2], data_frame = test_intl)

,Mean,Standard deviation,25%,50%,75%
Epi1,45.5,30.5,21.3,35.3,64.3
Proteinurieurine,1.0,1.4,0.2,0.5,1.3
Fosfaat,1.1,0.3,1.0,1.1,1.2
bmi,27.1,5.1,23.6,26.4,30.3
Hbmmolperliter,8.2,1.1,7.3,8.1,8.9
Calciumbloed,2.3,0.1,2.2,2.3,2.4
totaalcholesterol,4.8,1.3,3.9,4.7,5.5
age,55.6,15.5,47.0,57.0,68.0
esrd_lower,87.8,52.0,42.8,85.8,135.5


In [11]:
print('Median with range:\n')

# eGFR
egfr_desc = round(test_intl['Epi1'].describe(), 1)
print(f'eGFR:\n{egfr_desc["50%"]} [range, {egfr_desc["min"]} - {egfr_desc["max"]}]\n')

# Urinary protein excretion
protu_desc = round(test_intl['Proteinurieurine'].describe(), 1) #g/24hr
print(f'Urinary protein excretion:\n{protu_desc["50%"]} [range, {protu_desc["min"]} - {protu_desc["max"]}] g/24hrs\n')

# Phosphate
phosphate_desc = round(test_intl['Fosfaat'].describe(), 1) #mmol/l
print(f'Phosphate:\n{phosphate_desc["50%"]} [range, {phosphate_desc["min"]} - {phosphate_desc["max"]}] mmol/l\n')

# Hemoglobin
hb_desc = round(test_intl['Hbmmolperliter'].describe(), 1) #mmol/l
print(f'Hemoglobin:\n{hb_desc["50%"]} [range, {hb_desc["min"]} - {hb_desc["max"]}] mmol/l\n')

# Calcium
calc_desc = round(test_intl['Calciumbloed'].describe(), 1) #mmol/l
print(f'Calcium:\n{calc_desc["50%"]} [range, {calc_desc["min"]} - {calc_desc["max"]}] mmol/l\n')

# Total cholesterol
chol_desc = round(test_intl['totaalcholesterol'].describe(), 1)
print(f'Total cholesterol:\n{chol_desc["50%"]} [range, {chol_desc["min"]} - {chol_desc["max"]}] mmol/l')

Median with range:

eGFR:
35.3 [range, 5.9 - 134.7]

Urinary protein excretion:
0.5 [range, 0.0 - 8.7] g/24hrs

Phosphate:
1.1 [range, 0.6 - 2.1] mmol/l

Hemoglobin:
8.1 [range, 5.0 - 11.7] mmol/l

Calcium:
2.3 [range, 1.9 - 2.9] mmol/l

Total cholesterol:
4.7 [range, 1.9 - 10.1] mmol/l


## Calculate the incidence

In [12]:
import scipy.stats as st

def incidence(events, person_years, multiplier=100, alpha=0.05):
    
    # Events are all events and person years is all follow-up regardless of censoring
    rate = (events / person_years) * multiplier

    # PPF is percent point function
    z = st.norm.ppf(1 - alpha/2)

    if events > 0:
        factor = z / np.sqrt(events)
        lower = rate * np.exp(-factor)
        upper = rate * np.exp(+factor)
    else:
        lower, upper = 0.0, rate * np.exp(z/np.sqrt(events+1e-12))

    return rate, lower, upper

In [13]:
# Training data
total_events = train['ESRD'].sum()
total_person_years = train['esrd_lower'].sum() / 12
rate, lower_ci, upper_ci = incidence(total_events, total_person_years)
print('Training data:')
print(f'Incidence rate: {rate:.2f} (95% CI, {lower_ci:.2f}, {upper_ci:.2f}) per 100 patients/year\n')

# National validation
test['ESRD'] = (test['esrd_lower'] == test['esrd_upper']).astype(int)
total_events = test['ESRD'].sum()
total_person_years = test['esrd_lower'].sum() / 12
rate, lower_ci, upper_ci = incidence(total_events, total_person_years)
print('National validation:')
print(f'Incidence rate: {rate:.2f} (95% CI, {lower_ci:.2f}, {upper_ci:.2f}) per 100 patients/year\n')

# International validation
test_intl['ESRD'] = (test_intl['esrd_lower'] == test_intl['esrd_upper']).astype(int)
total_events = test_intl['ESRD'].sum()
total_person_years = test_intl['esrd_lower'].sum() / 12
rate, lower_ci, upper_ci = incidence(total_events, total_person_years)
print('International validation:')
print(f'Incidence rate: {rate:.2f} (95% CI, {lower_ci:.2f}, {upper_ci:.2f}) per 100 patients/year\n')

Training data:
Incidence rate: 5.69 (95% CI, 5.29, 6.12) per 100 patients/year

National validation:
Incidence rate: 8.37 (95% CI, 7.05, 9.95) per 100 patients/year

International validation:
Incidence rate: 5.25 (95% CI, 4.37, 6.31) per 100 patients/year

